In [126]:
# 필요한 라이브러리 import하기

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import optuna 
import lightgbm as lgb 
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score 
from sklearn.model_selection import cross_val_score, train_test_split
import os 

In [127]:
# train data와 test data 로드하기

train_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/wine_train_data.csv')
test_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/wine_test_data.csv')

In [128]:
train_X=train_data.drop('quality',axis=1)
train_y=train_data['quality']

new_test_data_X=test_data.drop('quality',axis=1)
new_test_data_y=test_data['quality']

In [129]:
train_y.value_counts()

0    684
1    595
Name: quality, dtype: int64

In [130]:
# class weight 적용하기 위해 y label의 비율을 구하기

ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [131]:
lgb=lgb.LGBMClassifier(learning_rate=0.03841283666283786, max_depth=3,n_estimators=50,scale_pos_weight=ratio,random_state=42)

In [132]:
# 모델 학습하기

lgb.fit(train_X,train_y)

LGBMClassifier(learning_rate=0.03841283666283786, max_depth=3, n_estimators=50,
               random_state=42, scale_pos_weight=1.149579831932773)

In [133]:
scores=cross_val_score(lgb,train_X,train_y,cv=5,scoring='roc_auc')
auc=scores.mean()
print(auc)

0.8283857896958698


In [134]:
# 모델의 성능(AUC)을 평가하기 위해 예측된 클래스가 불량일 확률 구하기

prob=lgb.predict_proba(new_test_data_X)[:,1]

In [135]:
# 모델 성능 평가함수 만들기

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.5,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [136]:
# 모델 평가하기

get_clf_prob(new_test_data_y,prob)

임계값:  0.5
오차행렬
[[129  42]
 [ 31 118]]
정확도: 0.7719, 정밀도: 0.7375, 재현율: 0.7919, F1:0.7716, AUC: 0.8403
